In [ ]:
import pandas as pd
import numpy as np
import os
import time
import glob
from random import sample

from scipy.io import wavfile, savemat
from scipy.fft import fft, fftfreq
from scipy import signal
from scipy.fft import fftshift
from scipy.io import loadmat
from sklearn.model_selection import train_test_split

from IPython.display import Audio, display
from IPython.display import clear_output

import librosa
import mne
import pymatreader
import yasa
import seaborn as sns
import pyxdf

from mne.time_frequency import psd_array_multitaper
from keras.callbacks import ModelCheckpoint
from mne.decoding import CSP
from scipy.integrate import simps
from yasa import sliding_window
from lazypredict.Supervised import LazyClassifier

import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from autoreject import AutoReject
 
# %matplotlib qt

%matplotlib inline
import plotly.io as pio
pio.renderers.default='notebook'

In [ ]:
# https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from EEGModels import EEGNet, ShallowConvNet, DeepConvNet

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Loading raw data and preprocessing it using MNE-python [Use EEGLAB instead]

In [ ]:
class Preprocess_Raw_EEG:
    def __init__(self):
        
        # Data constants
        self.sf = 1000
        self.nchan = 64
        self.ch_type = "eeg"
        self.channels = [f'E{n:1}' for n in range(1, self.nchan+1)]
        
        #Preprocessing constants
        self.low_pass = 40
        self.high_pass = 1
        self.notch = 50
        self.target_sf = 1000
        
        # Path Constants
        self.raw_eeg_path = "/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Raw"
        self.chanloc_path = "/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/ChannelLocations/EGIAmpServer64Chan.loc"
    
    def create_info(self):
        chans = mne.channels.read_custom_montage(self.chanloc_path)
        info = mne.create_info(channels, sfreq=self.sf, ch_types=self.ch_type)
        info.set_montage(chans)
        return info
    
    def create_events(self, stream):
        event_id_dict = {}
        ctr = 0
        events = stream[0][1]["time_series"]
        for mark in events:
            if mark[0] not in event_id_dict:
                if "ActionBeg" in mark[0]:
                    event_id_dict[mark[0]] = ctr
                    ctr += 1
        
        # creating array for epoching
        events = []
        time_stamps = stream[0][1]["time_stamps"]
        event_name = stream[0][1]["time_series"]
        event_name[0]
        for i in range(len(time_stamps)):
            if "ActionBeg" in event_name[i][0]:
                row = []
                row.append(int(time_stamps[i]))
                row.append(int(0))
                row.append(event_id_dict[event_name[i][0]])
                events.append(np.array(row))

        events = np.array(events)
        
        return event_id_dict, events
    
    def preprocess_data(self, stream):
        
        eeg_data = stream[0][0]["time_series"].T
        
        raw = mne.io.RawArray(eeg_data, self.create_info())
        
        # Filtering
        raw.filter(float(self.low_pass), float(self.high_pass), fir_design='firwin')
        raw.notch_filter(float(self.notch), method='fft')
        
        # Clean artifacts using ICA: remove 0, 1, and 2 independent components
        filt_raw = raw.copy()
        ica = ICA(n_components=10, max_iter='auto', random_state=97)
        ica.fit(filt_raw)
#         ica.exclude = [0, 1, 2]
        
        reconst_raw = raw.copy()
        ica.apply(reconst_raw)
        
        # Spherical Interpolation
        interp_raw = reconst_raw.copy().interpolate_bads(
            reset_bads=False, method=dict(eeg='MNE'), verbose=True
        )        
        
        # Re-referencing
        reref_raw = interp_raw.copy().set_eeg_reference(ref_channels='average')
        times_baseline_corr = np.array(reref_raw.to_data_frame().iloc[:, 0])
        
        # Creating epochs in the data and applying baseline correction trial-wise
        event_id_dict, events = self.create_events(stream)
        epochs = mne.Epochs(reref_raw, events, event_id = event_id_dict, tmin=-0.1, tmax=1.0)

        return epochs
    
    def get_ica_activations(self, stream):
        eeg_data = stream[0][0]["time_series"].T
        raw = mne.io.RawArray(eeg_data, self.create_info())
        
        # Filtering
        raw.filter(float(self.low_pass), float(self.high_pass), fir_design='firwin')
        raw.notch_filter(float(self.notch), method='fft')
        
        # Get ICA components and sphere
        filt_raw = raw.copy()
        ica = ICA(n_components=10, max_iter='auto', random_state=97)
        ica.fit(filt_raw)
        
        ica_act = np.matmul() ica.get_components() * ica.unmixing_matrix_

In [ ]:
preprocess = Preprocess_Raw_EEG()

# Loading Raw EEG data
temp_file = "P02.xdf"
stream = pyxdf.load_xdf(temp_file)

# Preprocessing
clean_epochs = preprocess.preprocess_data(stream)
clean_epochs

In [ ]:
raw_path = "/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Raw"
os.chdir(raw_path)

all_clean_epochs = []

for file in glob.glob("*.xdf"):
    print("##### ", file, "#####")
    
    # Reading raw eeg files using mne
    stream = pyxdf.load_xdf(file)
    
    # Preprocessing
    clean_epochs = preprocess.preprocess_data(stream)
    all_clean_epochs.append(clean_epochs)

In [ ]:
all_clean_epochs[0].to_data_frame()

# Independent Components for classification

In [ ]:
data_files

In [ ]:
def get_action_events(events):
    new_keys_dict = {}
    for key in events[1].keys():
        if "ActionBeg" in key:
            new_keys_dict[key] = events[1][key]
    new_event_times = [row for row in events[0] if row[2] in new_keys_dict.values()] 
    return (new_event_times, new_keys_dict)

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/PreprocessedData/ICs/")
print("Current directory: ", os.getcwd())
data_files = glob.glob("*.set")

eeg_df_main = pd.DataFrame()

for file_name in data_files:
    # Reading the set file
    print("Current file: ", file_name)
    arr = mne.io.read_raw_eeglab(file_name, verbose=0)
    eeg_data = arr.to_data_frame()
    events = mne.events_from_annotations(arr, verbose=0)
    
    # Extracting events of interest
    new_events = get_action_events(events)
    
    # Creating epochs
    epochs = mne.Epochs(arr, new_events[0], event_id=new_events[1], tmin=-0.1, tmax=1.0,  baseline=None, verbose=0)
    epoch_df = epochs.to_data_frame()
    
    # Transforming epoch dataframe to dataframe of interets
    all_epochs_df = pd.DataFrame()
    ctr = 1
    for n_epoch in range(n_epochs):
        eeg_only = epoch_df[epoch_df["epoch"] == n_epoch].reset_index().iloc[:, 4:].T
        label = epoch_df[epoch_df["epoch"] == n_epoch]["condition"].iloc[0]
        eeg_only["HandPos"] = label.split("-")[1]
        eeg_only["Hand"] = label.split("-")[2]
        eeg_only["Action"] = label.split("-")[3]
        eeg_only["Subject"] = file_name.split(".")[0]
        eeg_only["Component"] = [x[1:] for x in eeg_only.index]
        eeg_only["Trial"] = ctr
        eeg_only.reset_index()
        ctr += 1

        all_epochs_df = pd.concat([all_epochs_df, eeg_only])
        
    eeg_df_main = pd.concat([eeg_df_main, all_epochs_df])
    print(eeg_df_main.shape)

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/")
eeg_df_main.to_csv("ICs_v1.csv", index=None)

# Subject-wise classification based on independentcomponents

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/")
dataset = pd.read_csv("ICs_v1.csv")
dataset

In [ ]:
sub = "P02"
dataset[dataset["Subject"] == sub]

In [ ]:
# source : https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

clfs = [
    KNeighborsClassifier(2, n_jobs=-1),
    # SVC(kernel="linear", C=0.025),
#     SVC(gamma=2, C=1),
    # GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=2000),
    RandomForestClassifier(max_depth=2000, n_estimators=100, max_features=1, n_jobs=-1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]

names = [
    "KNN",
    # "Linear SVM",
#     "RBF SVM",
    # "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

clfs_mlp = [
    MLPClassifier(hidden_layer_sizes=(10, 20, 10), alpha=0.01, max_iter=1000),
    MLPClassifier(hidden_layer_sizes=(20, 40, 30), alpha=0.01, max_iter=1000),
    MLPClassifier(hidden_layer_sizes=(30, 60, 45), alpha=0.01, max_iter=1000),
    MLPClassifier(hidden_layer_sizes=(40, 80, 60), alpha=0.01, max_iter=1000),
    MLPClassifier(hidden_layer_sizes=(60, 120, 80), alpha=0.01, max_iter=1000),
    MLPClassifier(hidden_layer_sizes=(80, 140, 100), alpha=0.01, max_iter=1000),
]

names_mlp = [
    "MLP1",
    "MLP2",
    "MLP3",
    "MLP4",
    "MLP5",
    "MLP6",
]

In [ ]:
# subject-wise classification using different components one at a time
grasp_dict = {"Open": 0, "Close": 1}
sub = "P02"

all_results = []

for comp in range(1, 65):
    print("### Component ", comp, "###")
    X, y = [], []
    comp_data = dataset[(dataset["Component"] == comp) & (dataset["Subject"] == sub)]
    for trial in range(1, len(np.unique(dataset["Trial"]))+1 ):
        X.append(comp_data[comp_data["Trial"] == trial].iloc[0, :-6].to_numpy())
        y.append(grasp_dict[comp_data[comp_data["Trial"] == trial].iloc[:, -4].to_numpy()[0]])
    
    X = np.array(X)
    y = np.array(y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    
    results_comp = []
    for idx,clf in enumerate(clfs):
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        results_comp.append([comp, names[idx], score])
        print(names[idx], score)
    
    print()
    all_results += results_comp


In [ ]:
results_pp = []
for res in all_results:
    results_pp += res
results_pp

In [ ]:
results_df = pd.DataFrame(results_pp, columns=["Component", "Classifier", "Accuracy"])
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/v3/")
results_df.to_csv("P02-ML-all-Comp.csv", index=None)

In [ ]:
subjects = np.unique(dataset["Subject"])
trial = np.unique(dataset[dataset["Subject"] == "FP02"]["Trial"])
subjects, trial

In [ ]:
# Subject-wise and component-wise classification using Machine Learning

grasp_dict = {"Open": 0, "Close": 1}

comp_results = []
for comp in range(1, 65):
    print("### Component ", comp, "###")
    
    sub_results = []
    for sub in subjects:
        print("\t ### Subject ", sub, "###")
        trials = np.unique(dataset[dataset["Subject"] == sub]["Trial"])
        
        X, y = [], []
        for trial in trials:
            pc_eeg = dataset[
                (dataset["Subject"] == sub) & 
                (dataset["Component"] == comp) & 
                (dataset["Trial"] == trial)
            ]
            # print(pc_eeg.to_numpy()[0].shape)
            X.append(pc_eeg.iloc[0, :-6].to_numpy())
            y.append(grasp_dict[pc_eeg.iloc[:, -4].to_numpy()[0]])

        X, y = np.array(X), np.array(y)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

        results_comp = []
        for idx,clf in enumerate(clfs):
            clf.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            results_comp.append([comp, sub, names[idx], score])
            print("\t", names[idx], score)

        print()
        sub_results += results_comp
    comp_results += sub_results

In [ ]:
subject_wise_res_df = pd.DataFrame(comp_results, columns=["Component", "Subject", "Classifier", "Accuracy"])
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/v3/")
subject_wise_res_df.to_csv("AllSub-ML-AllComp.csv", index=None)

In [ ]:
# bad_participants = ["FP01", "FP10"]

In [ ]:
# Subject-wise and component-wise classification using EEGNET

grasp_dict = {"Open": 0, "Close": 1}
subjects = np.unique(dataset["Subject"])

sub_results = []
best_models = []
for sub in subjects:
    print("\t ### Subject ", sub, "###")
    trials = np.unique(dataset[dataset["Subject"] == sub]["Trial"])

    X, y = [], []
    for trial in trials:
        pc_eeg = dataset[(dataset["Subject"] == sub) & (dataset["Trial"] == trial)]
        
        X.append(pc_eeg.iloc[:, :-6].to_numpy())
        y.append(grasp_dict[pc_eeg.iloc[:, -4].to_numpy()[0]])

    X, y = np.array(X, dtype=np.float32), np.array(y)

    X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.1, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

    checkpoint_filepath = "/tmp/checkpoint/"
    checkpointer = ModelCheckpoint(
            filepath=checkpoint_filepath, 
            save_weights_only=True,
            monitor="val_accuracy",
            mode="max",
            save_best_only=True
    )

    modelEEGNET = EEGNet(nb_classes = 2, Chans = 64, Samples = 1101)
    modelEEGNET.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    fittedModel = modelEEGNET.fit(
        X_train, y_train, epochs=100, batch_size=128, validation_data=(X_eval, y_eval), 
        callbacks=[checkpointer], verbose=None
    )

    modelEEGNET.load_weights(checkpoint_filepath)
    y_pred = np.argmax(modelEEGNET.predict(X_test), axis=1)

    y_true = y_test
    test_acc = sum(y_pred == y_true) / len(y_true)
    print("Test accuracy: ", test_acc)

    sub_results += [sub, test_acc]
    best_models.append(modelEEGNET)


In [ ]:
np.reshape(sub_results, (15, 2))

In [ ]:
subject_wise_res_eegnet_df = pd.DataFrame(np.reshape(sub_results, (15, 2)), columns=["Subject", "Accuracy"])
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/v3/")
subject_wise_res_eegnet_df.to_csv("AllSub-EEGNET-AllComp.csv", index=None)

In [ ]:
subject_wise_res_eegnet_df

# Model Agnostic Meta Learning using Indepenent Components

# All subject classification using different components one at a time

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/")
dataset = pd.read_csv("ICs_v1.csv")
dataset

In [ ]:
grasp_dict = {"Open": 0, "Close": 1}

comp_res = []
for comp in range(1, 65):
    X, y = [], []
    print("Component ", comp)
    for sub in subjects:
        comp_data = dataset[(dataset["Component"] == comp) & (dataset["Subject"] == sub)]
        for trial in range(1, len(np.unique(dataset["Trial"]))+1 ):
            X.append(comp_data[comp_data["Trial"] == trial].iloc[0, :-6].to_numpy())
            y.append(grasp_dict[comp_data[comp_data["Trial"] == trial].iloc[:, -4].to_numpy()[0]])
    
    X = np.array(X)
    y = np.array(y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

    results = []
    for idx,clf in enumerate(clfs):
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        results.append([comp, names[idx], score])
        print(names[idx], score)
    
    comp_res += results


In [ ]:
comp_wise_res_df = pd.DataFrame(comp_res, columns=["Component", "Classifier", "Accuracy"])
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/v3/")
comp_wise_res_df.to_csv("Component-wise-Allsub.csv", index=None)

In [ ]:
os.chdir("/home/shivam/Desktop")
comp_wise_res_df.to_csv("Component-wise-Allsub.csv", index=None)

# Cross-subject classification using IC Cluster

# PSD Classification using entire band

In [ ]:
def get_action_events(events):
    new_keys_dict = {}
    for key in events[1].keys():
        if "ActionBeg" in key:
            new_keys_dict[key] = events[1][key]
    new_event_times = [row for row in events[0] if row[2] in new_keys_dict.values()] 
    return (new_event_times, new_keys_dict)

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/PreprocessedData/Preprocessed ICA/")
print("Current directory: ", os.getcwd())
data_files = glob.glob("*.set")

eeg_df_main = pd.DataFrame()

for file_name in data_files:
    # Reading the set file
    print("Current file: ", file_name)
    arr = mne.io.read_raw_eeglab(file_name, verbose=0)
    eeg_data = arr.to_data_frame()
    events = mne.events_from_annotations(arr, verbose=0)
    
    # Extracting events of interest
    new_events = get_action_events(events)
    
    # Creating epochs
    epochs = mne.Epochs(arr, new_events[0], event_id=new_events[1], tmin=-0.1, tmax=1.0,  baseline=None, verbose=0)
    epoch_df = epochs.to_data_frame()
    
    n_epochs = len(epochs)
    
    # Transforming epoch dataframe to dataframe of interets
    all_epochs_df = pd.DataFrame()
    ctr = 1
    for n_epoch in range(n_epochs):
        eeg_only = epoch_df[epoch_df["epoch"] == n_epoch].reset_index().iloc[:, 4:].T
        label = epoch_df[epoch_df["epoch"] == n_epoch]["condition"].iloc[0]
        eeg_only["HandPos"] = label.split("-")[1]
        eeg_only["Hand"] = label.split("-")[2]
        eeg_only["Action"] = label.split("-")[3]
        eeg_only["Subject"] = file_name.split(".")[0]
        eeg_only["Component"] = [x[1:] for x in eeg_only.index]
        eeg_only["Trial"] = ctr
        eeg_only.reset_index()
        ctr += 1

        all_epochs_df = pd.concat([all_epochs_df, eeg_only])
        
    eeg_df_main = pd.concat([eeg_df_main, all_epochs_df])
    print(eeg_df_main.shape)

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/")
eeg_df_main.to_csv("CLeanICAV1.csv", index=None)

In [ ]:
dataset = pd.read_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/CLeanICAV1.csv")
dataset

In [ ]:
# 64 channel EEG data with 500 samples
# Giving us X = N * 64 * 275 and y = N 
X, y = [], []
grasp_labels = {"Open": 0, "Close": 1}
nchan = 64
iterations = len(dataset) // nchan
for idx in range(iterations):
    start = idx * nchan
    end = (idx + 1) * nchan

    X.append(dataset.iloc[start: end, :1001])
    y.append(grasp_labels[dataset.iloc[start, -4]])

    
X = np.array(X)
y = np.array(y)

X.shape, y.shape

In [ ]:
# Multitaper method
def bandpower_multitaper(data, sf, method, band, relative=False):
    band = np.asarray(band)
    low, high = band

    if method == 'multitaper':
        psd_trial, freqs = psd_array_multitaper(data, sf, adaptive=True,
                                                normalization='full', verbose=0)
    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(psd_trial[idx_band], dx=freq_res)

    return bp


In [ ]:
def create_bandpower_features(epochs, frequency):
    bandpower_multitaper_EEG = []
    # Iterating over each subject [20]
    for epoch in epochs:
        # Iterating over each song per subject [30]
        bands_video = []
        no_channels = epoch.shape[0]
        input_brainwaves = epoch
        # Iterating over each channel [14]
        for k in range(no_channels):
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[8, 30], relative=False)) # Alpha and Beta frequency bands
#             bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
#                                                     band=[4, 7], relative=False))
#             bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
#                                                     band=[8, 13], relative=False))
#             bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
#                                                     band=[14, 30], relative=False))
#             bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
#                                                     band=[31, 50], relative=False))
      
        bandpower_multitaper_EEG.append(bands_video)

    bandpower_multitaper_EEG = np.array(bandpower_multitaper_EEG)
  
    return bandpower_multitaper_EEG

In [ ]:
sf = 1000
window = 1
psd_features = []
for x in X:
    epoch = sliding_window(x, sf=sf, window=window, step=window)[1]
    psd = create_bandpower_features(epoch, sf)
    if len(psd_features) == 0:
        psd_features = psd
    else:
        psd_features = np.concatenate((psd_features, psd))
    print(psd_features.shape)

In [ ]:
print(psd_features.shape)

In [ ]:
os.chdir('/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/IntData/')
np.save("pds_broadband_features.npy", psd_features)
np.save("pds_broadband_labels.npy", y)

In [ ]:
# os.chdir('/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/IntData/')
# psd_labels = []
# for label in y:
#     psd_labels += [label] * 1
# np.save("pds_broadband_labels.npy", y)

In [ ]:
os.chdir('/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/IntData/')
X, y = np.load("pds_broadband_features.npy"), np.load("pds_broadband_labels.npy")
X.shape, y.shape # ((1920, 320), (1920,))

In [ ]:
from sklearn.decomposition import PCA
# Take first 5 components using PCA 
pca = PCA(n_components=5)
X_r = pca.fit(X).transform(X)
X_r.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_r, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# source : https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

clfs = [
    KNeighborsClassifier(2),
    DecisionTreeClassifier(max_depth=2000),
    RandomForestClassifier(max_depth=2000, n_estimators=200, max_features="auto"),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]

names = [
    "Nearest Neighbors",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

clfs_mlp = [
    MLPClassifier(hidden_layer_sizes=(10, 20, 10), alpha=0.01, max_iter=1000),
    MLPClassifier(hidden_layer_sizes=(20, 40, 30), alpha=0.01, max_iter=1000),
    MLPClassifier(hidden_layer_sizes=(30, 60, 45), alpha=0.01, max_iter=1000),
    MLPClassifier(hidden_layer_sizes=(40, 80, 60), alpha=0.01, max_iter=1000),
    MLPClassifier(hidden_layer_sizes=(60, 120, 80), alpha=0.01, max_iter=1000),
    MLPClassifier(hidden_layer_sizes=(80, 140, 100), alpha=0.01, max_iter=1000),
]

names_mlp = [
    "MLP1",
    "MLP2",
    "MLP3",
    "MLP4",
    "MLP5",
    "MLP6",
]

In [ ]:
results = []
for idx,clf in enumerate(clfs_mlp):
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    results.append([names_mlp[idx], score])
    print(names_mlp[idx], score)

In [ ]:
# Something's wrong above ...